## Moat Brand Reporting Refresh

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [11]:
from rtf_utils.moat_utils import MoatTile
from essence.analytics.platform import securedcredentials as secure_creds
from rtf_utils.gcp_utils import BigQuery,CloudStorage
import warnings
from rtf_utils.bq_schema import moat_schema_dict
import json
import logging

In [12]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

In [13]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
print("Creds Loaded")
moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')
print("Token Loaded")

Creds Loaded
Token Loaded


In [15]:
start_date = '20191001'
end_date = '20191014'

In [1]:
with open('dags/brand_report_task_queue.json') as json_file:
    config_json = json.load(json_file)
    campaigns = config_json['campaigns']

NameError: name 'json' is not defined

In [16]:
## Define Moat Tasks for Stadia/Nest/Assistant
Q4 = [(2506,{'level1':22783112},['date','level1','level3']), ## disp stadia,
      (2506,{'level1':23221764},['date','level1','level3']), ## disp nest 1,
      (2506,{'level1':23197607},['date','level1','level3']), ## disp nest 2
      (2506,{'level1':23278016},['date','level1','level3']), ## disp assistant
      (2698,{'level1':22783112},['date','level1','level3']), ## vid stadia,
      (2698,{'level1':23221764},['date','level1','level3']), ## vid nest 1,
      (2698,{'level1':23197607},['date','level1','level3']), ## vid nest 2,
      (2698,{'level1':23278016},['date','level1','level3']),
      (6179366,{'level2':2604566125},['date','level2','level4']),## yt reserve - stadia
      (6178833,{'level1':1533586},['date','level1','level4']),## yt res dbm  - assistant
      (13386,{'level1':7492919005},['date','level1','level4']) ## stadia truview
 ]

# need access to 6196284

In [16]:
def get_moat_data(tile_id,start_date, end_date,level_filters=None,dimensions=None,**kwargs):
    tile = MoatTile(tile_id,level_filters,dimensions)
    local_filename = tile.get_data(start_date,end_date,moat_token)
    return local_filename

def upload_local_file(local_filename,dest_bucket,folder=None):
    gcs = CloudStorage(credentialsFromVault)
    if folder:
        blob_name = folder + "/" + local_filename
    else:
        blob_name = local_filename
    blob = gcs.upload_blob(dest_bucket, blob_name,local_filename,mode='filename')
    return blob

# Prototype WorkFlow

In [18]:
import os
import time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh'
#for x in bonito:
for i,x in enumerate([
    (2506,{'level1':22783112},['date','level1','level3']), ## disp stadia,
      (2506,{'level1':23221764},['date','level1','level3']), ## disp nest 1,
      (2506,{'level1':23197607},['date','level1','level3']), ## disp nest 2
      (2506,{'level1':23278016},['date','level1','level3'])]
    ):
    time.sleep(10)
    tile_id, level_filter, dimensions = x
    print(tile_id, level_filter, dimensions)
    filename = get_moat_data(tile_id,start_date,end_date,level_filter,dimensions)
    
    if filename:
        print("Stored at {}".format(filename))
    else:
        print("No Data")
        
        continue    
    
    try:
        file_uri = upload_local_file(filename,"rtf_staging",folder="brand_reporting")
        
        """file_uri = "gs://rtf_staging/" + file_uri.name
    
        print("Load Schema")
        #schema_dict = moat_schema_dict[tile_id]

        dest_table = "003_Moat_{}_{}".format(tile_id,i)

        print("Try BQ Upload: {}".format(dest_table))

        bq = BigQuery(credentialsFromVault)
        resp = bq.load_from_gcs("rtf_br_reporting",
                         file_uri,
                        dest_table,
                        #schema=schema_dict['schema'],
                        
                        extension='json')"""
    except Exception as e:
        print(e)
        continue

INFO:root:API Request Time


2506 {'level1': 22783112} ['date', 'level1', 'level3']


INFO:root:2506_22783112.json Saved


Stored at 2506_22783112.json


INFO:root:File uploaded as brand_reporting/2506_22783112.json.
INFO:root:API Request Time


...

INFO:root:File uploaded as brand_reporting/2506_23197607.json.
INFO:root:API Request Time


2506 {'level1': 23278016} ['date', 'level1', 'level3']


INFO:root:2506_23278016.json Saved


Stored at 2506_23278016.json


INFO:root:File uploaded as brand_reporting/2506_23278016.json.


In [50]:
file_uri = "gs://rtf_staging/brand_reporting/Brand Report - Upload File 2019-10-07T2046.csv"

dest_table = "003_DataLab_DailyPerformance"

bq.load_from_gcs("rtf_br_reporting",
                         file_uri,
                        dest_table)
                        #schema=schema_dict['schema']

In [19]:
f = {'level1':123345}

In [22]:
[*f.values()][0]

123345

In [19]:
"_".join([None,"Kyle",'Hey'])

TypeError: sequence item 0: expected str instance, NoneType found

In [20]:
"Kyle {}".format(None)

'Kyle None'

In [30]:
prefix = "we"
base = "Moat" 
suffix = None
extension = ".json"

l = [prefix,base,suffix]

"_".join([x for x in l if x ])  + extension

'we_Moat.json'

In [24]:
[x if x is not None for x in l ]

SyntaxError: invalid syntax (<ipython-input-24-a67a6d4ce7de>, line 1)

In [4]:

tasks = {
        2506:[
                ({'level1':22783112},['date','level1','level3']), ## disp stadia,
                ({'level1':23221764},['date','level1','level3']), ## disp nest 1,
                ({'level1':23197607},['date','level1','level3']), ## disp nest 2
                ({'level1':23278016},['date','level1','level3']), ## disp assistant
                ({'level1':22783112},['date','level1','level3']), ## vid stadia,
                ({'level1':23221764},['date','level1','level3']), ## vid nest 1,
                ({'level1':23197607},['date','level1','level3']), ## vid nest 2,
                ({'level1':23278016},['date','level1','level3'])
                ],
        6179366:[
                ({'level2':2604566125},['date','level2','level4'])
                ],## yt reserve - stadia
        6178833:[
            ({'level1':1533586},['date','level1','level4'])## yt res dbm  - assistant
                ],
        13386:[
            ({'level1':7492919005},['date','level1','level4'])
                ]}

In [11]:
for k,v in tasks.items():
    print(k)
    for request in v:
        a,b = request
        print(a)
        print(b)

6178833
{'level1': 1533586}
['date', 'level1', 'level4']
2506
{'level1': 22783112}
['date', 'level1', 'level3']
{'level1': 23221764}
['date', 'level1', 'level3']
{'level1': 23197607}
['date', 'level1', 'level3']
{'level1': 23278016}
['date', 'level1', 'level3']
{'level1': 22783112}
['date', 'level1', 'level3']
{'level1': 23221764}
['date', 'level1', 'level3']
{'level1': 23197607}
['date', 'level1', 'level3']
{'level1': 23278016}
['date', 'level1', 'level3']
13386
{'level1': 7492919005}
['date', 'level1', 'level4']
6179366
{'level2': 2604566125}
['date', 'level2', 'level4']


In [12]:
rand

NameError: name 'rand' is not defined

In [13]:
import random

In [19]:
random.randint(1,5)

1

In [ ]:
dd

In [22]:
import time
time.sleep(.5)

In [23]:
import json

In [31]:
tasks1 = {
        2506:[
                ({'level1':22783112},['date','level1','level3']), ## disp stadia,
                ({'level1':23221764},['date','level1','level3']), ## disp nest 1,
                ({'level1':23197607},['date','level1','level3']), ## disp nest 2
                ({'level1':23278016},['date','level1','level3']) ## disp assistant
            ]}

In [32]:
with open("tasks2.json",'w') as f:
    json.dump(tasks1,f)

In [33]:
schema = [
{'name':'date','type':'DATE','mode':'REQUIRED'},
{'name':'level1_id','type':'INTEGER','mode':'REQUIRED'},
{'name':'level1_label','type':'STRING', 'mode':'REQUIRED'},
{'name':'level3_id','type':'INTEGER', 'mode':'REQUIRED'},
{'name':'level3_label','type':'STRING', 'mode':'REQUIRED'},
{'name':'loads_unfiltered', 'type':'INTEGER', 'mode':'NULLABLE'},
{'name':'impressions_analyzed', 'type':'INTEGER', 'mode':'NULLABLE'}, 
{'name':'moat_score', 'type':'INTEGER', 'mode':'NULLABLE'}]

In [34]:
schema

[{'mode': 'REQUIRED', 'name': 'date', 'type': 'DATE'},
 {'mode': 'REQUIRED', 'name': 'level1_id', 'type': 'INTEGER'},
 {'mode': 'REQUIRED', 'name': 'level1_label', 'type': 'STRING'},
 {'mode': 'REQUIRED', 'name': 'level3_id', 'type': 'INTEGER'},
 {'mode': 'REQUIRED', 'name': 'level3_label', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'loads_unfiltered', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'impressions_analyzed', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'moat_score', 'type': 'INTEGER'}]

In [1]:
base_schema = [
{"name":"loads_unfiltered","type": "INTEGER","mode": "NULLABLE"),
{"name":"impressions_analyzed","type": "INTEGER","mode": "NULLABLE"),
{"name":"susp_human","type": "FLOAT","mode": "NULLABLE"),
{"name":"susp_valid","type": "FLOAT","mode": "NULLABLE"),
{"name":"human_and_viewable","type": "INTEGER","mode": "NULLABLE"),
{"name":"valid_and_viewable","type": "INTEGER","mode": "NULLABLE")
]

video_schema = [   
{"name":"reached_first_quart_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"reached_second_quart_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"reached_third_quart_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"reached_complete_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"player_audible_on_complete_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"player_visible_on_complete_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"player_audible_full_vis_half_time_sum","type": "INTEGER","mode": "NULLABLE"),
{"name":"player_vis_and_aud_on_complete_sum","type": "INTEGER","mode": "NULLABLE"),
]


moat_schemas = {

2506 :[
{"name":"date","type":"DATE","mode":"REQUIRED"},
{"name":"level1_id","type":"INTEGER","mode":"REQUIRED"},
{"name":"level1_label","type":"STRING", "mode":"REQUIRED"},
{"name":"level3_id","type":"INTEGER", "mode":"REQUIRED"},
{"name":"level3_label","type":"STRING", "mode":"REQUIRED"}] + base_schema + [
{"name":"human_and_viewable", "type":"INTEGER", "mode":"NULLABLE"},
{"name":"valid_and_viewable", "type":"INTEGER", "mode":"NULLABLE"},
{"name":"moat_score", "type":"INTEGER", "mode":"NULLABLE"},
{"name":"iva", "type":"INTEGER", "mode":"NULLABLE"}
],   

6179366 : [
{"name":"date","type": "DATE","mode": "REQUIRED"),
{"name":"level2_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level2_label","type": "STRING","mode": "REQUIRED"),
{"name":"level4_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level4_label","type": "STRING","mode": "REQUIRED")] + base_schema + [
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"),
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"),
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE")] + video_schema,


6178833 : [
{"name":"date","type": "DATE","mode": "REQUIRED"),
{"name":"level2_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level2_label","type": "STRING","mode": "REQUIRED"),
{"name":"level4_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level4_label","type": "STRING","mode": "REQUIRED")} + base_schema + [
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"),
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"),
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE")
] + video_schema,

13386: [
{"name":"date","type": "DATE","mode": "REQUIRED"),
{"name":"level2_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level2_label","type": "STRING","mode": "REQUIRED"),
{"name":"level4_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level4_label","type": "STRING","mode": "REQUIRED")] + base_schema + [
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"),
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"),
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE")
] + video_schema,

2698 :[
{"name":"date","type": "DATE","mode": "REQUIRED"), # make required
{"name":"level1_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level1_label","type": "STRING","mode": "REQUIRED"),
{"name":"level3_id","type": "INTEGER","mode": "REQUIRED"),
{"name":"level3_label","type": "STRING","mode": "REQUIRED")] + base_schema + 
[
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"),
{"name":"avg_real_estate_perc","type": "FLOAT","mode": "NULLABLE"),
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"),
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE")] + video_schema

}


SyntaxError: invalid syntax (<ipython-input-1-41f01388f33e>, line 2)

In [5]:
base_schema = [
{"name":"loads_unfiltered","type": "INTEGER","mode": "NULLABLE"},
{"name":"impressions_analyzed","type": "INTEGER","mode": "NULLABLE"},
{"name":"susp_human","type": "FLOAT","mode": "NULLABLE"},
{"name":"susp_valid","type": "FLOAT","mode": "NULLABLE"},
{"name":"human_and_viewable","type": "INTEGER","mode": "NULLABLE"},
{"name":"valid_and_viewable","type": "INTEGER","mode": "NULLABLE"}
]

video_schema = [   
{"name":"reached_first_quart_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"reached_second_quart_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"reached_third_quart_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"reached_complete_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"player_audible_on_complete_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"player_visible_on_complete_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"player_audible_full_vis_half_time_sum","type": "INTEGER","mode": "NULLABLE"},
{"name":"player_vis_and_aud_on_complete_sum","type": "INTEGER","mode": "NULLABLE"},
]


moat_schemas = {

2506 :[
{"name":"date","type":"DATE","mode":"REQUIRED"},
{"name":"level1_id","type":"INTEGER","mode":"REQUIRED"},
{"name":"level1_label","type":"STRING", "mode":"REQUIRED"},
{"name":"level3_id","type":"INTEGER", "mode":"REQUIRED"},
{"name":"level3_label","type":"STRING", "mode":"REQUIRED"}] + base_schema + [
{"name":"human_and_viewable", "type":"INTEGER", "mode":"NULLABLE"},
{"name":"valid_and_viewable", "type":"INTEGER", "mode":"NULLABLE"},
{"name":"moat_score", "type":"INTEGER", "mode":"NULLABLE"},
{"name":"iva", "type":"INTEGER", "mode":"NULLABLE"}
],   

6179366 : [
{"name":"date","type": "DATE","mode": "REQUIRED"},
{"name":"level2_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level2_label","type": "STRING","mode": "REQUIRED"},
{"name":"level4_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level4_label","type": "STRING","mode": "REQUIRED"}] + base_schema + [
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"},
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"},
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"}] + video_schema,


6178833 : [
{"name":"date","type": "DATE","mode": "REQUIRED"},
{"name":"level2_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level2_label","type": "STRING","mode": "REQUIRED"},
{"name":"level4_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level4_label","type": "STRING","mode": "REQUIRED"}] + base_schema + [
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"},
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"},
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"}
] + video_schema,

13386: [
{"name":"date","type": "DATE","mode": "REQUIRED"},
{"name":"level2_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level2_label","type": "STRING","mode": "REQUIRED"},
{"name":"level4_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level4_label","type": "STRING","mode": "REQUIRED"}] + base_schema + [
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"},
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"},
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"}
] + video_schema,

2698 :[
{"name":"date","type": "DATE","mode": "REQUIRED"}, # make required
{"name":"level1_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level1_label","type": "STRING","mode": "REQUIRED"},
{"name":"level3_id","type": "INTEGER","mode": "REQUIRED"},
{"name":"level3_label","type": "STRING","mode": "REQUIRED"}] + base_schema + 
[
{"name":"_5_sec_in_view_impressions","type": "INTEGER","mode": "NULLABLE"},
{"name":"avg_real_estate_perc","type": "FLOAT","mode": "NULLABLE"},
{"name":"susp_human_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"},
{"name":"susp_valid_and_inview_gm_meas_sum","type": "FLOAT","mode": "NULLABLE"}] + video_schema
    }

In [6]:
from rtf_utils.moat_utils import moat_schemas

In [7]:
moat_schemas

{2506: [{'mode': 'REQUIRED', 'name': 'date', 'type': 'DATE'},
  {'mode': 'REQUIRED', 'name': 'level1_id', 'type': 'INTEGER'},
  {'mode': 'REQUIRED', 'name': 'level1_label', 'type': 'STRING'},
  {'mode': 'REQUIRED', 'name': 'level3_id', 'type': 'INTEGER'},
  {'mode': 'REQUIRED', 'name': 'level3_label', 'type': 'STRING'},
  {'mode': 'NULLABLE', 'name': 'loads_unfiltered', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'impressions_analyzed', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'susp_human', 'type': 'FLOAT'},
  {'mode': 'NULLABLE', 'name': 'susp_valid', 'type': 'FLOAT'},
  {'mode': 'NULLABLE', 'name': 'human_and_viewable', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'valid_and_viewable', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'human_and_viewable', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'valid_and_viewable', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'moat_score', 'type': 'INTEGER'},
  {'mode': 'NULLABLE', 'name': 'iva', 'type': 'IN